<a href="https://colab.research.google.com/github/SadrikA78/shlak/blob/main/Textattack_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Установка TextAttack**

In [3]:
!git clone https://github.com/QData/TextAttack.git


fatal: destination path 'TextAttack' already exists and is not an empty directory.


In [6]:
%cd /content/TextAttack
!pip install -e .

/content/TextAttack
Obtaining file:///content/TextAttack
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 110.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 116.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Импорт модели**

In [7]:
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Данные для эксперимента**

In [8]:
# dataset: An iterable of (text, ground_truth_output) pairs.
#0 means the review is negative
#1 means the review is positive
custom_dataset = [
    ('I hate this movie', 0), #A negative comment, with a negative label
    ('I hate this movie', 1), #A negative comment, with a positive label
    ('I love this movie', 0), #A positive comment, with a negative label
    ('I love this movie', 1), #A positive comment, with a positive label
]

**Атака на тональность**

In [9]:
from textattack import Attack
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.goal_functions import UntargetedClassification
from textattack.transformations import WordSwapEmbedding
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

# We'll use untargeted classification as the goal function.
goal_function = UntargetedClassification(model_wrapper)
# We'll to use our WordSwapEmbedding as the attack transformation.
transformation = WordSwapEmbedding()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We'll use the Greedy search method
search_method = GreedySearch()
# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:12<00:00, 38.5MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpiveeurzo.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


**Вывод** (все норм)

In [10]:
for example, label in custom_dataset:
    result = attack.attack(example, label)
    print(result.__str__(color_method='ansi'))

0 (99%) --> 1 (81%)

I hate this movie

did hateful this footage
0 (99%) --> [38:5:240m[SKIPPED]

I hate this movie
1 (96%) --> [38:5:240m[SKIPPED]

I love this movie
1 (96%) --> 0 (99%)

I love this movie

I iove this movie
